In [2]:
import numpy as np
import pandas as pd

import wikipedia
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

import time

In [21]:
stroke_count_html = requests.get('https://en.wikipedia.org/wiki/Radical_214')
# stroke_count_html

In [22]:
stroke_count_bs = BeautifulSoup(stroke_count_html.text)
# stroke_count_bs

In [26]:
stroke_count_table = stroke_count_bs.find_all(name = 'table')[5]
# stroke_count_table

In [43]:
stroke_count_table_tbody = stroke_count_table.find_all(name = 'tbody')
# stroke_count_table_tbody

In [44]:
stroke_count_table_tr = stroke_count_table_tbody[0].find_all(name = 'tr')
stroke_count_table_tr[17].text # get 17 strokes column 

'17 strokes\n214 龠\n'

In [50]:
print(stroke_count_table_tr[1].text.split('\n'))

['1 stroke', '1 一', '2 丨', '3 丶', '4 丿', '5 乙', '6 亅', '']


In [57]:
indices = dict()
for i in range(1, 18):
    row_list = stroke_count_table_tr[i].text.split('\n')
    indices[i] = []
    
    for j in range(1, len(row_list) - 1):
        num, rad = row_list[j].split()
        num = int(num)
        
        item = dict()
        item[num] = rad
        indices[i].append(item)


print(indices)

{1: [{1: '一'}, {2: '丨'}, {3: '丶'}, {4: '丿'}, {5: '乙'}, {6: '亅'}], 2: [{7: '二'}, {8: '亠'}, {9: '人'}, {10: '儿'}, {11: '入'}, {12: '八'}, {13: '冂'}, {14: '冖'}, {15: '冫'}, {16: '几'}, {17: '凵'}, {18: '刀'}, {19: '力'}, {20: '勹'}, {21: '匕'}, {22: '匚'}, {23: '匸'}, {24: '十'}, {25: '卜'}, {26: '卩'}, {27: '厂'}, {28: '厶'}, {29: '又'}], 3: [{30: '口'}, {31: '囗'}, {32: '土'}, {33: '士'}, {34: '夂'}, {35: '夊'}, {36: '夕'}, {37: '大'}, {38: '女'}, {39: '子'}, {40: '宀'}, {41: '寸'}, {42: '小'}, {43: '尢'}, {44: '尸'}, {45: '屮'}, {46: '山'}, {47: '巛'}, {48: '工'}, {49: '己'}, {50: '巾'}, {51: '干'}, {52: '幺'}, {53: '广'}, {54: '廴'}, {55: '廾'}, {56: '弋'}, {57: '弓'}, {58: '彐'}, {59: '彡'}, {60: '彳'}], 4: [{61: '心'}, {62: '戈'}, {63: '戶'}, {64: '手'}, {65: '支'}, {66: '攴'}, {67: '文'}, {68: '斗'}, {69: '斤'}, {70: '方'}, {71: '无'}, {72: '日'}, {73: '曰'}, {74: '月'}, {75: '木'}, {76: '欠'}, {77: '止'}, {78: '歹'}, {79: '殳'}, {80: '毋'}, {81: '比'}, {82: '毛'}, {83: '氏'}, {84: '气'}, {85: '水'}, {86: '火'}, {87: '爪'}, {88: '父'}, {89: '爻'}, {90: '爿'},